In [1]:
import pandas as pd

file_name=[
    '01','02','03','04','05','06','07','08','09','10',
    '11','12','13','14','15','16','17','18','19','20',
    '21','22','23','24','25','26','27','28','29','30',
    '31','32','33','34','35','36','37','38','39','40',
    '41','42','43','44','45','46','47','48','49','50',
    '51','52','53','54','55','56','57','58','59','60']

data=[]

for i in range(len(file_name)):
    data.append(pd.read_csv('../data/'+ file_name[i] +'_tracks.csv'))

# Standard deviation of speed

# Create Pull Request


In [2]:
print(data[0])

        frame    id       x      y  width  height  xVelocity  yVelocity  \
0           1     1  362.26  21.68   4.85    2.12      40.85       0.00   
1           2     1  363.73  21.68   4.85    2.12      40.87       0.00   
2           3     1  365.27  21.68   4.85    2.12      40.88       0.00   
3           4     1  366.83  21.68   4.85    2.12      40.89       0.00   
4           5     1  368.42  21.68   4.85    2.12      40.90       0.01   
...       ...   ...     ...    ...    ...     ...        ...        ...   
348745  22535  1047  366.97  12.54   6.37    3.13     -35.63       0.19   
348746  22536  1047  365.55  12.55   6.37    3.13     -35.66       0.20   
348747  22537  1047  364.12  12.56   6.37    3.13     -35.69       0.20   
348748  22538  1047  362.69  12.57   6.37    3.13     -35.72       0.21   
348749  22539  1047  361.26  12.58   6.37    3.13     -35.75       0.21   

        xAcceleration  yAcceleration  ...  precedingXVelocity  precedingId  \
0                0.30